In [4]:
from __future__ import print_function, division
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [5]:
# Create functions to apply to the dataframe
def clean_url(row):                             #Extract URL from Link
    return row['link'].find("a")['href']

def clean_text(row):                            #Extract Text
    return row.text

def get_date(row):                              #Extract Date from Link
    if row.text == '-':
        return ''
    else: return row.find("a")['href'][47:57]   

def clean_num(row):                             #Extract Number and strip non numerical parts
    return row.text.replace('$','').replace(',','')

In [8]:
#Scrape Data from Website                                             
#years = [3,range(2008,2018)]                           #The first number is the # of pages(-1) for each year to scrape
years = [3,range(1988,1998)]                           #the second is the range of years you want to scrape

for year in years[1]:
    movie_data = [] 
    for page in range(1,years[0]):                     #the top number in the range sets how many sequential pages you want to scrape
        response = requests.get('https://www.boxofficemojo.com/yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year))
        print(response.status_code)

        webpage = response.text
        soup = BeautifulSoup(webpage, "lxml")
        tables_soup = soup.find_all('table')[6]         #this is the soup that contains just the table I want
        rows_soup = tables_soup.find_all('tr')          #This is a soup that contains all of the rows in the table 

        for j in range(2,102):                          #this sets the range for rows, which the first movie is at 
                                                        #index 2 and the last is at index 101
            data_soup = rows_soup[j].find_all('td')     #turn each row into a soup object of data
            row_data = []                               #start an empty list
            for i in range(0,8):
                row_data.append(data_soup[i])           #add each data item to the list
            movie_data.append(row_data)                 #at the end of a row, add that row list to the full movie list
            
    #turn big movie list into DataFrame
    movie_df = pd.DataFrame(movie_data)             

    #Rename columns in the dataframe to the names in the table/chart
    movie_df = movie_df.rename({0:'rank', 1: "link", 2: "studio", 3: "total_box", 
                                    4:'max_sites', 5: "open_box", 6:'open_sites', 
                                    7:'open_date', 8: 'close_date' }, axis='columns')#
    
    #Apply definitions to clean data
    movie_df['url'] = movie_df.apply(clean_url,axis=1)                #Clean URL
    movie_df['title'] = movie_df['link'].apply(clean_text)            #Clean Title
    movie_df['rank'] = movie_df['rank'].apply(clean_text)             #Clean rank
    movie_df['studio'] = movie_df['studio'].apply(clean_text)         #Clean Studio Name
    movie_df['total_box'] = movie_df['total_box'].apply(clean_num)    #Clean Total Box Office
    movie_df['max_sites'] = movie_df['max_sites'].apply(clean_num)    #Clean Max Number of Theaters
    movie_df['open_box'] = movie_df['open_box'].apply(clean_num)      #Clean Opening Weekend Box Office
    movie_df['open_sites'] = movie_df['open_sites'].apply(clean_num)  #Clean Opening Weekend NUmber of Theaters
    movie_df['open_date'] = movie_df['open_date'].apply(get_date)     #Clean Opening Date
    #movie_df['close_date'] = movie_df['close_date'].apply(clean_text)#Clean Closing Date

    movie_df.drop(['link'], axis=1,inplace = True)
    
    #Save CSV with filename 'year_movies.csv'
    movie_df.to_csv('../04_Data/{}_movies.csv'.format(year))  

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
